In [13]:
import tensorflow as tf

from network.cnn_definition_paper import rgb255_to_obj_net 
from keras.layers import Lambda

from xnp.xnp_tf_layers import MtMCreation, MtMEvaluation
from reverse_op import PODestarisation

from data_handling.model_info import load_model_info
from data_handling import DataLoader

bop_path  = '/tf/notebooks/datasets'
dataset = 'tless'
dataset_path = f'{bop_path}/{dataset}'

print(f'Dataset path:   {dataset_path}')
print(f'GPUs Available: {tf.config.list_physical_devices("GPU")}')

results_path = 'results'

foreign_info = 'scene_maskrcnn_detections_syn+real_info.json'

test = ['test_primesense']

xyDim = 112
strides = 2

root_dir = [f'{dataset_path}/{d}' for d in test]


# import json
# import os

# foreign_info_path = f'{dataset_path}/bop23_default_detections_for_task1/gdrnppdet-pbrreal/gdrnppdet-pbrreal_tless-test_e112ecb4-7f56-4107-8a21-945bc7661267.json'
# with open(foreign_info_path, 'r') as f:
#     foreign_info_data = json.load(f)

# for rd in root_dir:
#     for root, sub_dirs, files in os.walk(rd):
#         for sd in sub_dirs:
#             dir = f'{root}/{sd}'
#             scene_foreign_info = f'{dir}/{foreign_info}'
                
#             filtered_data = [entry for entry in foreign_info_data if entry['scene_id'] == int(sd)]
#             with open(scene_foreign_info, 'w') as f:
#                 json.dump(filtered_data, f, indent=4)
#         break

Dataset path:   /tf/notebooks/datasets/tless
GPUs Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [20]:
def totuplething(x1, x2, x4, x7):
    return ((x1, x2, x4, x7), x7)

for oiu in range(1,2):
    model_info = load_model_info(dataset_path, oiu, verbose=1)

    trained_cnn = tf.keras.models.load_model(f'{dataset_path}/saved_models/csl_o{oiu}_trainable_layers')

    rgb_input, depth_input, cam_K, coord_K = trained_cnn.inputs
    star_representation, dash_representation, w_px, w_d, seg = trained_cnn.outputs

    segmentation = Lambda(lambda x: tf.cast(x>0.75, x.dtype))(seg)
    w_px = Lambda(lambda x: x[0][...,tf.newaxis,:] * x[1][...,tf.newaxis])([w_px, segmentation])
    w_d = Lambda(lambda x: x[0][...,tf.newaxis,:] * x[1][...,tf.newaxis])([w_d, segmentation])

    po_image = PODestarisation(model_info,amount_of_instances = 1)(star_representation, dash_representation, segmentation)
    po_image = Lambda(lambda x: x[0][...,tf.newaxis,:] * x[1][...,tf.newaxis])([po_image, segmentation])
    MtM = MtMCreation("po", strides)((po_image, w_px, cam_K, coord_K))
    pred_T_, pred_Omega = MtMEvaluation()(MtM)

    depth = Lambda(lambda x: x[:,::strides,::strides])(depth_input)
    MtM_d = MtMCreation("depth")((po_image, depth , w_d))
    pred_T_d, pred_Omega = MtMEvaluation()(tf.keras.layers.Add()([MtM, MtM_d]))

    #pred_model = tf.keras.Model(trained_cnn.inputs, (pred_T_, pred_T_d))
    pred_model = tf.keras.Model(trained_cnn.inputs, (rgb_input, star_representation, dash_representation, segmentation))

    test_foreign_data = DataLoader.load_foreign_data([f'{dataset_path}/{d}' for d in test ], foreign_info, oiu)
    print(f'Found TEST foreign data for {len(test_foreign_data)} occurencies of object {oiu}, where {len([d for d in test_foreign_data if "primesense" in d["root"]])} origined from primesense.')

    res = pred_model.predict(DataLoader.Dataset(test_foreign_data, xyDim, test_mode=True).batch(1).prefetch(20).map(totuplething), verbose=1)

    rowwisestr = 'scene_id,im_id,obj_id,score,R,t,time\n'
    rowwisestr_d = 'scene_id,im_id,obj_id,score,R,t,time\n'
    for tdi, td in enumerate(test_foreign_data):
        pose_ = res[0][tdi][0]
        pose_d = res[1][tdi][0]
        rowwisestr += f'{int(td["root"][-6:])},{int(td["file_name"])},{oiu},0.5,{" ".join(list(pose_[:3,:3].flatten().astype(str)))},{" ".join(list(pose_[:3,3].flatten().astype(str)))},-1\n'
        rowwisestr_d += f'{int(td["root"][-6:])},{int(td["file_name"])},{oiu},0.5,{" ".join(list(pose_d[:3,:3].flatten().astype(str)))},{" ".join(list(pose_d[:3,3].flatten().astype(str)))},-1\n'
    
    f = open(f"{results_path}/new-paper-{oiu}-cosypose-frame-rgb_tless-test.csv","w")
    f.write(rowwisestr)
    f.close()

    f = open(f"{results_path}/new-paper-{oiu}-cosypose-frame-rgb-d_tless-test.csv","w")
    f.write(rowwisestr_d)
    f.close()

model_info for object 1:
diameter : 63.5151
mins : [-17.4958 -17.4958 -30.6   ]
maxs : [17.4958 17.4958 30.6   ]
symmetries_discrete : []
symmetries_continuous : True
destarring as symmetries_continuous


100%|██████████| 20/20 [00:00<00:00, 130.81it/s]


/tf/notebooks/datasets/tless/test_primesense/000017/scene_maskrcnn_detections_syn+real_info.json
/tf/notebooks/datasets/tless/test_primesense/000009/scene_maskrcnn_detections_syn+real_info.json
/tf/notebooks/datasets/tless/test_primesense/000015/scene_maskrcnn_detections_syn+real_info.json
/tf/notebooks/datasets/tless/test_primesense/000002/scene_maskrcnn_detections_syn+real_info.json
/tf/notebooks/datasets/tless/test_primesense/000018/scene_maskrcnn_detections_syn+real_info.json
/tf/notebooks/datasets/tless/test_primesense/000006/scene_maskrcnn_detections_syn+real_info.json
/tf/notebooks/datasets/tless/test_primesense/000011/scene_maskrcnn_detections_syn+real_info.json
/tf/notebooks/datasets/tless/test_primesense/000005/scene_maskrcnn_detections_syn+real_info.json
/tf/notebooks/datasets/tless/test_primesense/000016/scene_maskrcnn_detections_syn+real_info.json
/tf/notebooks/datasets/tless/test_primesense/000014/scene_maskrcnn_detections_syn+real_info.json
/tf/notebooks/datasets/tless/t

IndexError: index 3 is out of bounds for axis 1 with size 3

In [21]:
import numpy as np
np.save("rgb_input.npy", res[0])
np.save("star.npy", res[1])
np.save("dash.npy", res[2])
np.save("seg.npy", res[3])

In [ ]:
combined_results = 'scene_id,im_id,obj_id,score,R,t,time\n'
for oiu in range(1,31):
    f = open(f"{results_path}/new-paper-{oiu}-cosypose-frame-rgb_tless-test.csv","r")
    combined_results += f.read()[len('scene_id,im_id,obj_id,score,R,t,time\n'):]
    f.close()
    
f = open(f"{results_path}/new-paper-1-30-cosypose-frame-rgb_tless-test.csv","w")
f.write(combined_results)
f.close()

combined_results = 'scene_id,im_id,obj_id,score,R,t,time\n'
for oiu in range(1,31):
    f = open(f"{results_path}/new-paper-{oiu}-cosypose-frame-rgb-d_tless-test.csv","r")
    combined_results += f.read()[len('scene_id,im_id,obj_id,score,R,t,time\n'):]
    f.close()
    
f = open(f"{results_path}/new-paper-1-30-cosypose-frame-rgb-d_tless-test.csv","w")
f.write(combined_results)
f.close()
        